<a href="https://colab.research.google.com/github/d-kleine/Advent_of_HayStack/blob/main/07_Arize_Phoenix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Haystack - Day 7
_Make a copy of this Colab to start_

Santa collapsed in his chair in a huff. "What's wrong?" asked Mrs Claus.

"There's just too many toys to check and not enough time! Christmas is almost here!"

"Well can't you just check some of them?"

"I wish it were that easy! But my elves make so many different toys, and we have to make sure every kid gets the right one!"

Elf Jane couldn't help overhearing from the next room. She was a regular attendee at the local north pole hackathon, and thought she might have a solution. She'd learned a lot about evaluation recently, and thought she could build an LLM Judge to help.

**For this challenge, you need to help Elf Jane and complete the code cells with `#TODO` text**

 <img src='https://github.com/Jgilhuly/phoenix-assets/blob/main/images/socal/advent-of-haystack-1.jpeg?raw=true' width=500px>

## Installation

In [1]:
# !pip install -q arize-phoenix==6.1.0 haystack-ai==2.7.0 openinference-instrumentation-haystack==0.1.13 httpx<0.28

## Data

Elf Jane started by checking out the big elf database of christmas wishlists (aka the BEDCW).

In [2]:
children = [
    {'name': 'Timmy', 'age': 7, 'likes': 'Lego', 'dislikes': 'Vegetables', 'list': 'nice'},
    {'name': 'Tommy', 'age': 9, 'likes': 'Sports Equipment', 'dislikes': 'Reading', 'list': 'naughty'},
    {'name': 'Tammy', 'age': 8, 'likes': 'Art Supplies', 'dislikes': 'Loud Noises', 'list': 'nice'},
    {'name': 'Tina', 'age': 6, 'likes': 'Science Kits', 'dislikes': 'Spicy Food', 'list': 'nice'},
    {'name': 'Toby', 'age': 10, 'likes': 'Video Games', 'dislikes': 'Early Mornings', 'list': 'nice'},
    {'name': 'Tod', 'age': 5, 'likes': 'Musical Instruments', 'dislikes': 'Bath Time', 'list': 'nice'},
    {'name': 'Todd', 'age': 8, 'likes': 'Remote Control Cars', 'dislikes': 'Homework', 'list': 'naughty'},
    {'name': 'Tara', 'age': 7, 'likes': 'Magic Sets', 'dislikes': 'Thunder', 'list': 'nice'},
    {'name': 'Teri', 'age': 9, 'likes': 'Building Blocks', 'dislikes': 'Broccoli', 'list': 'nice'},
    {'name': 'Trey', 'age': 6, 'likes': 'Board Games', 'dislikes': 'Bedtime', 'list': 'nice'},
    {'name': 'Tyler', 'age': 8, 'likes': 'Action Figures', 'dislikes': 'Cleaning', 'list': 'nice'},
    {'name': 'Tracy', 'age': 7, 'likes': 'Dolls', 'dislikes': 'Dark', 'list': 'nice'},
    {'name': 'Tony', 'age': 9, 'likes': 'Chemistry Sets', 'dislikes': 'Dentist', 'list': 'nice'},
    {'name': 'Theo', 'age': 6, 'likes': 'Puzzles', 'dislikes': 'Shots', 'list': 'nice'},
    {'name': 'Terry', 'age': 10, 'likes': 'Model Trains', 'dislikes': 'Chores', 'list': 'naughty'},
    {'name': 'Tessa', 'age': 5, 'likes': 'Stuffed Animals', 'dislikes': 'Time Out', 'list': 'nice'},
    {'name': 'Troy', 'age': 8, 'likes': 'Robots', 'dislikes': 'Naps', 'list': 'nice'},
    {'name': 'Talia', 'age': 7, 'likes': 'Craft Kits', 'dislikes': 'Spinach', 'list': 'nice'},
    {'name': 'Tyson', 'age': 9, 'likes': 'Microscopes', 'dislikes': 'Cold', 'list': 'nice'},
    {'name': 'Tatum', 'age': 6, 'likes': 'Drawing Sets', 'dislikes': 'Shots', 'list': 'nice'},
]

In [3]:
len(children)

20

# 1. Adding Tracing 📝

Elf Jane knew that the elves were busy, and didn't always log their toy making process. She knew that she'd first need to trace the toy making process using Arize Phoenix.

In [4]:
from getpass import getpass

from phoenix.otel import register
from openinference.instrumentation.haystack import HaystackInstrumentor

# TODO: Add Phoenix tracing with Haystack: https://docs.arize.com/phoenix/tracing/integrations-tracing/haystack
# There are many ways to launch Phoenix - the simplest way for this example is to use the "Notebook" option

import os
from getpass import getpass

# Add Phoenix API Key for tracing
PHOENIX_API_KEY = getpass("PHOENIX_API_KEY")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

# configure the Phoenix tracer
tracer_provider = register(
  project_name="toy_finder", # Default is 'default'
  endpoint="https://app.phoenix.arize.com/v1/traces",
  set_global_tracer_provider=False
)

HaystackInstrumentor().instrument(tracer_provider=tracer_provider)

OpenTelemetry Tracing Details
|  Phoenix Project: toy_finder
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



# 2. Trace Toy Making Process 🚂

With tracing in place, Elf Jane had some of her closest elf friends build a batch of toys she could trace.

⭐️ Feel free to replace `OpenAIChatGenerator` with other [ChatGenerators](https://docs.haystack.deepset.ai/docs/generators) supported in Haystack

In [5]:
import json

with open("config.json", "r") as config_file:
    os.environ["OPENAI_API_KEY"] = json.load(config_file)

In [6]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.builders import ChatPromptBuilder
from haystack import Pipeline

messages = [
    ChatMessage.from_system("You are a toy maker elf. Your job is to make toys for the nice kids on the nice list. If the child is on the naughty list, give them a 'Rabbit R1'. {{name}} is on the {{list}} list"),
    ChatMessage.from_user("Create a toy for {{name}} that they will like. {{name}} is {{age}} years old and likes {{likes}} and dislikes {{dislikes}}."),
]

builder = ChatPromptBuilder(messages)
chat_generator = OpenAIChatGenerator(model="gpt-4o-mini")

pipeline = Pipeline()
pipeline.add_component("builder", builder)
pipeline.add_component("chat_generator", chat_generator)

pipeline.connect("builder", "chat_generator")

def make_toy(child):
    return pipeline.run({"builder":{**child}})["chat_generator"]["replies"]

In [7]:
for child in children:
    make_toy(child)

# 3. Evaluate Toy Correctness 🔬

Elf Jane was now ready to evaluate the toys she made. She knew that she could use an LLM Judge to evaluate whether the toys matched the child's wishlist. She started by building a judge.

In [8]:
llm_judge_prompt = """
Evaluate the toy for this child, based on their likes and dislikes.

All children on the naughty list get a 'Rabbit R1'. Any other toy given to a naughty child is incorrect.

Respond with a single word: 'correct' or 'incorrect'. Also include a short explanation for your answer.

Description of the child: {description}
Toy: {toy}

*****
Example output:
label: 'correct'
explanation: 'The toy is a Lego set, which is one of the child's likes.'
*****
"""

In [9]:
import phoenix as px

# TODO: Download the traces from Phoenix
# HINT: https://docs.arize.com/phoenix/evaluation/how-to-evals/evaluating-phoenix-traces#download-trace-dataset-from-phoenix

# Download the traces from Phoenix
spans_df = px.Client().get_spans_dataframe(project_name="toy_finder")

# Display the first few rows of the dataframe
spans_df

c:\Users\dk\anaconda3\envs\haystack\Lib\site-packages\phoenix\utilities\client.py:34: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.output.mime_type,attributes.input.value,attributes.openinference.span.kind,attributes.input.mime_type,attributes.llm.token_count.prompt,attributes.llm.model_name,attributes.llm.token_count.completion,attributes.llm.input_messages,attributes.llm.token_count.total,attributes.llm.output_messages
context.span_id,,,,,,,,,,,,,,,,,,,,,
5f49f2406cf1209b,ChatPromptBuilder (builder),CHAIN,7d2154f4f064e4b7,2024-12-17 14:41:24.667695+00:00,2024-12-17 14:41:24.669237+00:00,OK,,[],5f49f2406cf1209b,741898ba0b5b3b4e733aa6634f2be808,...,application/json,"{""template"": null, ""template_variables"": null,...",CHAIN,application/json,NaN,None,NaN,None,NaN,None
4a3c634813b2d73a,OpenAIChatGenerator (chat_generator),LLM,7d2154f4f064e4b7,2024-12-17 14:41:25.075670+00:00,2024-12-17 14:41:28.973888+00:00,OK,,[],4a3c634813b2d73a,741898ba0b5b3b4e733aa6634f2be808,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,83.0,gpt-4o-mini-2024-07-18,208.0,"[{'message.role': 'system', 'message.content':...",291.0,"[{'message.role': 'assistant', 'message.conten..."
7d2154f4f064e4b7,Pipeline,CHAIN,None,2024-12-17 14:41:24.666143+00:00,2024-12-17 14:41:29.171983+00:00,OK,,[],7d2154f4f064e4b7,741898ba0b5b3b4e733aa6634f2be808,...,application/json,"{""data"": {""builder"": {""name"": ""Timmy"", ""age"": ...",CHAIN,application/json,NaN,None,NaN,None,NaN,None
efa62ed4dfd49738,ChatPromptBuilder (builder),CHAIN,123071a02d7e0555,2024-12-17 14:41:29.377215+00:00,2024-12-17 14:41:29.378754+00:00,OK,,[],efa62ed4dfd49738,65854dcfb497c80f3f01058d4501dd33,...,application/json,"{""template"": null, ""template_variables"": null,...",CHAIN,application/json,NaN,None,NaN,None,NaN,None
9482fbf13a9382e3,OpenAIChatGenerator (chat_generator),LLM,123071a02d7e0555,2024-12-17 14:41:29.518193+00:00,2024-12-17 14:41:31.835158+00:00,OK,,[],9482fbf13a9382e3,65854dcfb497c80f3f01058d4501dd33,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,80.0,gpt-4o-mini-2024-07-18,116.0,"[{'message.role': 'system', 'message.content':...",196.0,"[{'message.role': 'assistant', 'message.conten..."
123071a02d7e0555,Pipeline,CHAIN,None,2024-12-17 14:41:29.377215+00:00,2024-12-17 14:41:31.971466+00:00,OK,,[],123071a02d7e0555,65854dcfb497c80f3f01058d4501dd33,...,application/json,"{""data"": {""builder"": {""name"": ""Tommy"", ""age"": ...",CHAIN,application/json,NaN,None,NaN,None,NaN,None
09cac719c5480736,ChatPromptBuilder (builder),CHAIN,16a67dae0759aa47,2024-12-17 14:41:32.143945+00:00,2024-12-17 14:41:32.145457+00:00,OK,,[],09cac719c5480736,a96488f7f7e06fecd4cfb20add6327ed,...,application/json,"{""template"": null, ""template_variables"": null,...",CHAIN,application/json,NaN,None,NaN,None,NaN,None
7e105d75b99a58f7,OpenAIChatGenerator (chat_generator),LLM,16a67dae0759aa47,2024-12-17 14:41:32.281399+00:00,2024-12-17 14:41:36.456066+00:00,OK,,[],7e105d75b99a58f7,a96488f7f7e06fecd4cfb20add6327ed,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,82.0,gpt-4o-mini-2024-07-18,171.0,"[{'message.role': 'system', 'message.content':...",253.0,"[{'message.role': 'assistant', 'message.conten..."
16a67dae0759aa47,Pipeline,CHAIN,None,2024-12-17 14:41:32.143945+00:00,2024-12-17 14:41:36.647138+00:00,OK,,[],16a67dae0759aa47,a96488f7f7e06fecd4cfb20add6327ed,...,application/json,"{""data"": {""builder"": {""name"": ""Tammy"", ""age"": ...",CHAIN,application/json,NaN,None,NaN,None,NaN,None


In [10]:
filtered_df = spans_df[(spans_df['span_kind'] == 'LLM')]
filtered_df

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.output.mime_type,attributes.input.value,attributes.openinference.span.kind,attributes.input.mime_type,attributes.llm.token_count.prompt,attributes.llm.model_name,attributes.llm.token_count.completion,attributes.llm.input_messages,attributes.llm.token_count.total,attributes.llm.output_messages
context.span_id,,,,,,,,,,,,,,,,,,,,,
4a3c634813b2d73a,OpenAIChatGenerator (chat_generator),LLM,7d2154f4f064e4b7,2024-12-17 14:41:25.075670+00:00,2024-12-17 14:41:28.973888+00:00,OK,,[],4a3c634813b2d73a,741898ba0b5b3b4e733aa6634f2be808,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,83.0,gpt-4o-mini-2024-07-18,208.0,"[{'message.role': 'system', 'message.content':...",291.0,"[{'message.role': 'assistant', 'message.conten..."
9482fbf13a9382e3,OpenAIChatGenerator (chat_generator),LLM,123071a02d7e0555,2024-12-17 14:41:29.518193+00:00,2024-12-17 14:41:31.835158+00:00,OK,,[],9482fbf13a9382e3,65854dcfb497c80f3f01058d4501dd33,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,80.0,gpt-4o-mini-2024-07-18,116.0,"[{'message.role': 'system', 'message.content':...",196.0,"[{'message.role': 'assistant', 'message.conten..."
7e105d75b99a58f7,OpenAIChatGenerator (chat_generator),LLM,16a67dae0759aa47,2024-12-17 14:41:32.281399+00:00,2024-12-17 14:41:36.456066+00:00,OK,,[],7e105d75b99a58f7,a96488f7f7e06fecd4cfb20add6327ed,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,82.0,gpt-4o-mini-2024-07-18,171.0,"[{'message.role': 'system', 'message.content':...",253.0,"[{'message.role': 'assistant', 'message.conten..."
14302e18b6e52d92,OpenAIChatGenerator (chat_generator),LLM,5a36dcc0210eb9e2,2024-12-17 14:41:36.954366+00:00,2024-12-17 14:41:40.641059+00:00,OK,,[],14302e18b6e52d92,a6361d399731c235e13ed8df4d993392,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,82.0,gpt-4o-mini-2024-07-18,213.0,"[{'message.role': 'system', 'message.content':...",295.0,"[{'message.role': 'assistant', 'message.conten..."
5c2567ea9d790ae1,OpenAIChatGenerator (chat_generator),LLM,f1176a4292b089f5,2024-12-17 14:41:41.086145+00:00,2024-12-17 14:41:45.660110+00:00,OK,,[],5c2567ea9d790ae1,ac782e5d5bf1ad987e327c419f88da68,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,83.0,gpt-4o-mini-2024-07-18,235.0,"[{'message.role': 'system', 'message.content':...",318.0,"[{'message.role': 'assistant', 'message.conten..."
4fa81a1384f79694,OpenAIChatGenerator (chat_generator),LLM,915c9b7b7bd5ee68,2024-12-17 14:41:46.102940+00:00,2024-12-17 14:41:50.882627+00:00,OK,,[],4fa81a1384f79694,e6834017f36b55b6f73734209a55f342,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,81.0,gpt-4o-mini-2024-07-18,235.0,"[{'message.role': 'system', 'message.content':...",316.0,"[{'message.role': 'assistant', 'message.conten..."
5ea37515c03289ea,OpenAIChatGenerator (chat_generator),LLM,45ce1797ef65ca1f,2024-12-17 14:41:51.325552+00:00,2024-12-17 14:41:54.080861+00:00,OK,,[],5ea37515c03289ea,b2d44bbfd321a509dfe6bdc730de6700,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,81.0,gpt-4o-mini-2024-07-18,98.0,"[{'message.role': 'system', 'message.content':...",179.0,"[{'message.role': 'assistant', 'message.conten..."
d22a0b486085959e,OpenAIChatGenerator (chat_generator),LLM,bb18f8a31147e7d7,2024-12-17 14:41:54.566544+00:00,2024-12-17 14:41:58.365720+00:00,OK,,[],d22a0b486085959e,98062ca06b3fe744ec6138f037b4026f,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,80.0,gpt-4o-mini-2024-07-18,239.0,"[{'message.role': 'system', 'message.content':...",319.0,"[{'message.role': 'assistant', 'message.conten..."
d2bd0f628f1e247f,OpenA

In [11]:
input_messages = filtered_df['attributes.llm.input_messages']
output_messages = filtered_df['attributes.llm.output_messages']

# Function to extract 'message.content'
def extract_message_content(messages):
    return [message['message.content'] for message in messages]

# Extract content from input and output messages
input_contents = input_messages.apply(extract_message_content)
output_contents = output_messages.apply(extract_message_content)

In [15]:
str(input_contents.iloc[0]).split('. ')[-1].strip()

"Timmy is 7 years old and likes Lego and dislikes Vegetables.']"

In [28]:
" ".join(output_contents.iloc[0])

'For Timmy, who loves Lego, I would create a special Lego-themed toy set called the "Lego Adventure Castle." This set would include:\n\n1. **Castle Structure**: A large, multi-story castle that Timmy can build and customize, featuring towers, walls, and roofs that can be rearranged.\n\n2. **Minifigures**: A set of knights, wizards, and mythical creatures like dragons and fairies to populate the castle and inspire imaginative play.\n\n3. **Interactive Features**: Fun elements like a drawbridge that raises and lowers, secret passages, and a treasure chest filled with “gold” bricks.\n\n4. **Expansions Packs**: Additional themed expansion packs (like a dragon\'s cave or a wizard\'s tower) that Timmy can collect and integrate with his castle.\n\n5. **Building Challenges**: An instruction booklet with fun building challenges and story prompts to encourage creativity.\n\nThis Lego Adventure Castle would provide hours of engaging and creative play, perfectly suited for Timmy\'s interests!'

In [19]:
# Extract the last sentence from each input
extracted_descriptions = input_contents.apply(lambda x: str(x).split('. ')[-1].strip())
print(extracted_descriptions)

context.span_id
4a3c634813b2d73a    Timmy is 7 years old and likes Lego and dislik...
9482fbf13a9382e3    Tommy is 9 years old and likes Sports Equipmen...
7e105d75b99a58f7    Tammy is 8 years old and likes Art Supplies an...
14302e18b6e52d92    Tina is 6 years old and likes Science Kits and...
5c2567ea9d790ae1    Toby is 10 years old and likes Video Games and...
4fa81a1384f79694    Tod is 5 years old and likes Musical Instrumen...
5ea37515c03289ea    Todd is 8 years old and likes Remote Control C...
d22a0b486085959e    Tara is 7 years old and likes Magic Sets and d...
d2bd0f628f1e247f    Teri is 9 years old and likes Building Blocks ...
9a98d11262c117f9    Trey is 6 years old and likes Board Games and ...
319b6e2a7cf42e48    Tyler is 8 years old and likes Action Figures ...
c839ce2ad189dc41    Tracy is 7 years old and likes Dolls and disli...
4595ab17e638a611    Tony is 9 years old and likes Chemistry Sets a...
610586cca5433bc9    Theo is 6 years old and likes Puzzles and disl...
6bc5

In [29]:
extracted_toys = output_contents.apply(lambda x: " ".join(x))
print(extracted_toys)

context.span_id
4a3c634813b2d73a    For Timmy, who loves Lego, I would create a sp...
9482fbf13a9382e3    Since Tommy is on the naughty list, I'll be gi...
7e105d75b99a58f7    For Tammy, I will create a special "Art Advent...
14302e18b6e52d92    For Tina, I would create a **"Fun Science Lab ...
5c2567ea9d790ae1    For Toby, I recommend creating an amazing **Ga...
4fa81a1384f79694    For Tod, who is 5 years old and loves musical ...
5ea37515c03289ea    Since Todd is on the naughty list, he'll recei...
d22a0b486085959e    For Tara, I will create a magical “Enchanted M...
d2bd0f628f1e247f    For Teri, I would create a special set of "Mag...
9a98d11262c117f9    For Trey, I’ll create a fun board game called ...
319b6e2a7cf42e48    For Tyler, an 8-year-old who loves action figu...
c839ce2ad189dc41    For Tracy, I suggest creating a beautiful **"M...
4595ab17e638a611    For Tony, who is 9 years old and loves Chemist...
610586cca5433bc9    For Theo, I will create a **Mystery Puzzle Adv...
6bc5

In [30]:
import pandas as pd

prompts_df = pd.DataFrame({'description': extracted_descriptions, 'toy': extracted_toys})

# Display the new DataFrame
prompts_df.head()

,description,toy
context.span_id,,
4a3c634813b2d73a,Timmy is 7 years old and likes Lego and dislik...,"For Timmy, who loves Lego, I would create a sp..."
9482fbf13a9382e3,Tommy is 9 years old and likes Sports Equipmen...,"Since Tommy is on the naughty list, I'll be gi..."
7e105d75b99a58f7,Tammy is 8 years old and likes Art Supplies an...,"For Tammy, I will create a special ""Art Advent..."
14302e18b6e52d92,Tina is 6 years old and likes Science Kits and...,"For Tina, I would create a **""Fun Science Lab ..."
5c2567ea9d790ae1,Toby is 10 years old and likes Video Games and...,"For Toby, I recommend creating an amazing **Ga..."


In [34]:
from phoenix.evals import (
    llm_classify,
    OpenAIModel # can swap for another model supported by Phoenix or run open-source models through LiteLLM and Ollama: https://docs.arize.com/phoenix/evaluation/evaluation-models
)

import nest_asyncio
nest_asyncio.apply()

# TODO: Evaluate the traces with the LLM Judge
# HINT: https://docs.arize.com/phoenix/evaluation/how-to-evals/bring-your-own-evaluator#categorical-llm_classify

rails = ["incorrect", "correct"]
eval_results = llm_classify(
    dataframe=prompts_df,
    template=llm_judge_prompt,
    model=OpenAIModel(model="gpt-4o-mini"),
    rails=rails
)

llm_classify |          | 0/20 (0.0%) | ⏳ 00:00<? | ?it/s

In [35]:
eval_results["score"] = eval_results["label"].apply(lambda x: 1 if x == "correct" else 0)
eval_results

,label,exceptions,execution_status,execution_seconds,score
context.span_id,,,,,
4a3c634813b2d73a,incorrect,[],COMPLETED,2.107999,0
9482fbf13a9382e3,correct,[],COMPLETED,1.292697,1
7e105d75b99a58f7,incorrect,[],COMPLETED,0.600581,0
14302e18b6e52d92,incorrect,[],COMPLETED,0.587444,0
5c2567ea9d790ae1,incorrect,[],COMPLETED,0.572722,0
4fa81a1384f79694,incorrect,[],COMPLETED,0.569657,0
5ea37515c03289ea,correct,[],COMPLETED,1.797209,1
d22a0b486085959e,incorrect,[],COMPLETED,1.592569,0
d2bd0f628f1e247f,incorrect,[],COMPLETED,0.979518,0


In [36]:
from phoenix.trace import SpanEvaluations

# TODO: Upload results into Phoenix
# HINT: https://docs.arize.com/phoenix/evaluation/how-to-evals/evaluating-phoenix-traces#download-trace-dataset-from-phoenix

eval_results["score"] = eval_results["score"].astype(int)
eval_results["label"] = eval_results["label"].astype(str)

px.Client().log_evaluations(SpanEvaluations(eval_name="toy", dataframe=eval_results))

c:\Users\dk\anaconda3\envs\haystack\Lib\site-packages\phoenix\utilities\client.py:34: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(


# 4. View the results in the Arize Phoenix UI 🐦‍🔥

And just like that, Elf Jane had saved Santa hours of time and made sure every kid got the right toy!

In Phoenix, she could see "correct" and "incorrect" labels on all the traces, and even see the explanations for each label!

She couldn't wait to show Santa, and all her friends at the hackathon.

 <img src='https://github.com/Jgilhuly/phoenix-assets/blob/main/images/socal/advent-of-haystack-2.jpeg?raw=true' width=500px>